High-resolution profile of neoantigen-specific TCR activation links moderate stimulation to resilience and sustained tumor control

Franziska Füchsl*, Johannes Untch*, Vladyslav Kavaka, Sebastian Jarosch, Carolin Vogelsang, Niklas de Andrade Krätzig, Dario Gosmann, Roland Rad, Dirk Busch, Eduardo Beltrán, Eva Bräunlein#, Angela M. Krackhardt#

Analysis by: Vladyslav Kavaka (vladyslav.kavaka@med.uni-muenchen.de), Supervision: Eduardo Beltrán (eduardo.beltran@med.uni-muenchen.de)

In [3]:
#For the R part of the code
sessionInfo()
set.seed(1)
.libPaths()

R version 4.0.5 (2021-03-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /home/INIM/vladyslav.kavaka/miniconda3/envs/azimuth/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] RColorBrewer_1.1-3             SeuratDisk_0.0.0.9019         
 [3] bonemarrowref.SeuratData_1.0.0 SeuratData_0.2.2              
 [5] ggupset_0.3.0                  ggrastr_1.0.1                 
 [7] celldex_1.0.0                  Nebulosa_1.0.2                
 [9] harmony_0.1.0                  Rcpp_1.0.8                    
[11] enrichR_3.0       

[1] "/home/INIM/vladyslav.kavaka/miniconda3/envs/azimuth/lib/R/library"

In [7]:
#For the python based part
session_info.show()

# Imports

In [2]:
library(devtools)
library(vctrs)
library(Seurat)
library(dplyr)
library(Matrix)
library(tidyr)
library(limma)
library(ggplot2)
library(ggthemes)
library(patchwork)
library(gprofiler2)
library(ggrepel)
library(scales)
library(ggthemes)
library(purrr)
library(MAST)
library(qpcR)
library(enrichR)
library(harmony)
library(Nebulosa)
library(celldex)
library(ggrastr)
library(ggupset)
library(SeuratData)
library(SeuratDisk)
library(RColorBrewer)

In [ ]:
options(repr.plot.width=11, repr.plot.height=11)

# Read 10X data for the first sample

In [ ]:
## Read 10X data:
matrix_dir = "~/pathway_to_data/outs/filtered_feature_bc_matrix/"

In [ ]:
list.files(matrix_dir)

In [ ]:
data <- Read10X(data.dir = matrix_dir)

In [ ]:
str(data)

In [ ]:
## Create Seurat object
pbmc <- CreateSeuratObject (counts = data, min.cells = 3, min.features = 200, project = "pbmc_1")

In [ ]:
pbmc

In [ ]:
# The number of features and UMIs (nFeature_RNA and nCount_RNA) are automatically calculated for every object by Seurat.
# For non-UMI data, nCount_RNA represents the sum of the non-normalized values within a cell
# We calculate the percentage of mitochondrial features here and store it in object metadata as `percent.mito`.
# We use raw count data since this represents non-transformed and non-log-normalized counts
# The % of UMI mapping to MT-features is a common scRNA-seq QC metric.
mito.features <- grep(pattern = "^MT-", x = rownames(x = pbmc), value = TRUE)
percent.mito <- Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts'))

In [ ]:
# The [[ operator can add columns to object metadata, and is a great place to stash QC stats
pbmc[['percent.mito']] <- percent.mito
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3, pt.size = 0.000001)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used for anything 
# calculated by the object, i.e. columns in object metadata, PC scores etc.
# Since there is a rare subset of cells with an outlier level of high mitochondrial percentage
# and also low UMI content, we filter these as well
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "percent.mito")

In [ ]:
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

In [ ]:
# We filter out cells that have unique feature counts over 5,000 or less than 200
pbmc <- subset(x = pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 6000 & percent.mito < '0.18')

In [ ]:
pbmc

## Add TCR information of the first sample

In [ ]:
#First we need to create TCR file for the first sample in the list
tcr <- read.csv(paste('~/pathway_to_data/cellranger_vdj/outs/G1=stimulated/outs/', "filtered_contig_annotations.csv", sep=""))
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste('~/pathway_to_TCR_data/cellranger_vdj/outs/G1=stimulated/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"
head(tcr)
tcr.combined <- tcr

In [ ]:
#divide in TRA and TRB subset:
for (k in 1:nrow(tcr.combined)){
  if(startsWith(tcr.combined$TCR1[k], 'TRB:')){
    tcr.combined$TCR1B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR1[k], 'TRA:')){
    tcr.combined$TCR1A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR2[k], 'TRB:')){
    tcr.combined$TCR2B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2B[k] <- 'FALSE'}
        if(startsWith(tcr.combined$TCR2[k], 'TRA:')){
    tcr.combined$TCR2A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR3[k], 'TRB:')){
    tcr.combined$TCR3B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3B[k] <- 'FALSE'}
     if(startsWith(tcr.combined$TCR3[k], 'TRA:')){
    tcr.combined$TCR3A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR4[k], 'TRB:')){
    tcr.combined$TCR4B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR4[k], 'TRA:')){
    tcr.combined$TCR4A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4A[k] <- 'FALSE'}
}

In [ ]:
tcr.combined$TCR1 <- NULL
tcr.combined$TCR2 <- NULL
tcr.combined$TCR3 <- NULL
tcr.combined$TCR4 <- NULL
head(tcr.combined)
tail(tcr.combined)

In [ ]:
nrow(tcr.combined)

In [ ]:
write.csv(tcr.combined, file = './tcr_1.csv')

In [ ]:
pbmc <- AddMetaData(object = pbmc, metadata = tcr.combined)

In [ ]:
md = pbmc@meta.data # First, let's get the meta data
i <- sapply(md, is.factor) # Identify all factor variables in your data
md[i] <- lapply(md[i], as.character) # Convert factors to character variables
md[is.na(md)] <- "FALSE" # Replace NA with "FALSE"
md[i] <- lapply(md[i], as.factor) # Convert character columns back to factors
pbmc@meta.data = md #Insert it back

In [ ]:
pbmc_1 <- pbmc

# Read 10X data for the second sample

In [ ]:
## Read 10X data:
matrix_dir = "~/pathway_to_data/outs/filtered_feature_bc_matrix/"

In [ ]:
list.files(matrix_dir)

In [ ]:
data <- Read10X(data.dir = matrix_dir)

In [ ]:
str(data)

In [ ]:
## Create Seurat object
pbmc <- CreateSeuratObject (counts = data, min.cells = 3, min.features = 200, project = "pbmc_2")

In [ ]:
pbmc

In [ ]:
# The number of features and UMIs (nFeature_RNA and nCount_RNA) are automatically calculated for every object by Seurat.
# For non-UMI data, nCount_RNA represents the sum of the non-normalized values within a cell
# We calculate the percentage of mitochondrial features here and store it in object metadata as `percent.mito`.
# We use raw count data since this represents non-transformed and non-log-normalized counts
# The % of UMI mapping to MT-features is a common scRNA-seq QC metric.
mito.features <- grep(pattern = "^MT-", x = rownames(x = pbmc), value = TRUE)
percent.mito <- Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts'))

In [ ]:
# The [[ operator can add columns to object metadata, and is a great place to stash QC stats
pbmc[['percent.mito']] <- percent.mito
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3, pt.size = 0.000001)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used for anything 
# calculated by the object, i.e. columns in object metadata, PC scores etc.
# Since there is a rare subset of cells with an outlier level of high mitochondrial percentage
# and also low UMI content, we filter these as well
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "percent.mito")

In [ ]:
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

In [ ]:
# We filter out cells that have unique feature counts over 5,000 or less than 200
pbmc <- subset(x = pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 6000 & percent.mito < '0.18')

In [ ]:
pbmc

## Add TCR information of the first sample

In [ ]:
#First we need to create TCR file for the first sample in the list
tcr <- read.csv(paste('~/pathway_to_data/outs/', "filtered_contig_annotations.csv", sep=""))
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste('~/pathway_to_data/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"
head(tcr)
tcr.combined <- tcr

In [ ]:
#divide in TRA and TRB subset:
for (k in 1:nrow(tcr.combined)){
  if(startsWith(tcr.combined$TCR1[k], 'TRB:')){
    tcr.combined$TCR1B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR1[k], 'TRA:')){
    tcr.combined$TCR1A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR2[k], 'TRB:')){
    tcr.combined$TCR2B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2B[k] <- 'FALSE'}
        if(startsWith(tcr.combined$TCR2[k], 'TRA:')){
    tcr.combined$TCR2A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR3[k], 'TRB:')){
    tcr.combined$TCR3B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3B[k] <- 'FALSE'}
     if(startsWith(tcr.combined$TCR3[k], 'TRA:')){
    tcr.combined$TCR3A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR4[k], 'TRB:')){
    tcr.combined$TCR4B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR4[k], 'TRA:')){
    tcr.combined$TCR4A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4A[k] <- 'FALSE'}
}

In [ ]:
tcr.combined$TCR1 <- NULL
tcr.combined$TCR2 <- NULL
tcr.combined$TCR3 <- NULL
tcr.combined$TCR4 <- NULL
head(tcr.combined)
tail(tcr.combined)

In [ ]:
nrow(tcr.combined)

In [ ]:
write.csv(tcr.combined, file = './tcr_2.csv')

In [ ]:
pbmc <- AddMetaData(object = pbmc, metadata = tcr.combined)

In [ ]:
md = pbmc@meta.data # First, let's get the meta data
i <- sapply(md, is.factor) # Identify all factor variables in your data
md[i] <- lapply(md[i], as.character) # Convert factors to character variables
md[is.na(md)] <- "FALSE" # Replace NA with "FALSE"
md[i] <- lapply(md[i], as.factor) # Convert character columns back to factors
pbmc@meta.data = md #Insert it back

In [ ]:
pbmc_2 <- pbmc

# Subset only T cells

In [ ]:
colnames(pbmc.integrated@meta.data)

In [ ]:
Idents(object = pbmc.integrated) <- "TCR_clonotype_id"
tcells <- subset(x = pbmc.integrated, idents = "FALSE", invert = TRUE)
tcells

# Integration of T cells with 2000 features

In [ ]:
DefaultAssay(tcells) <- 'RNA'

In [ ]:
pbmc.list <- SplitObject(object = tcells, split.by = "orig.ident")

In [ ]:
pbmc.list

In [ ]:
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV",  x = rownames(x = pbmc), value = TRUE) #remove the markers for TCR variable chains before performing integration

# normalize and identify variable features for each dataset independently
pbmc.list <- lapply(X = pbmc.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2500)
})

for (i in 1:length(pbmc.list)){
     VariableFeatures(pbmc.list[[i]]) <- VariableFeatures(object = pbmc.list[[i]])[!(VariableFeatures(object = pbmc.list[[i]])%in%markers.remove)]
}

In [ ]:
# select features that are repeatedly variable across datasets for integration
features <- SelectIntegrationFeatures(object.list = pbmc.list, nfeatures = 2000)

In [ ]:
pbmc.anchors <- FindIntegrationAnchors(object.list = pbmc.list, anchor.features = features)

In [ ]:
tcells.integrated <- IntegrateData(anchorset = pbmc.anchors, dims = 1:30)

In [ ]:
tcells.integrated
tcells <- tcells.integrated
tcells

In [ ]:
saveRDS(tcells, file = 'tcells_integrated_withoutumap.rds')

# Find VariableFeatures with vst method, scaling, dimensional reduction and clustering:

In [ ]:
DefaultAssay(tcells) <- 'integrated'
tcells

In [ ]:
tcells <- FindVariableFeatures(tcells, selection.method = "vst", nfeatures = 2000)
length(x = VariableFeatures(object = tcells))
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS",  x = rownames(x = tcells), value = TRUE)
VariableFeatures(object = tcells) <- VariableFeatures(object = tcells)[!(VariableFeatures(object = tcells)%in%markers.remove)]
length(VariableFeatures(object = tcells))

In [ ]:
tcells <- ScaleData(tcells, features = VariableFeatures(object = tcells), vars.to.regress = c("nCount_RNA", "percent.mito"))
tcells <- RunPCA(tcells, features = VariableFeatures(object = tcells))

In [ ]:
Idents(tcells) <- 'orig.ident'
DimPlot(tcells, reduction = 'pca', label = TRUE)

In [ ]:
ElbowPlot(object = tcells, ndims = 50)

In [ ]:
DimHeatmap(object = tcells, dims = 1, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 2, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 3, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 4, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 5, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 6, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 7, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 8, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 9, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 10, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 11, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 12, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 13, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 14, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 15, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 16, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 17, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 18, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 19, cells = 500, balanced = TRUE)
DimHeatmap(object = tcells, dims = 20, cells = 500, balanced = TRUE)

# Cluster the cells

In [ ]:
DefaultAssay(tcells) <- 'integrated'
tcells

In [ ]:
tcells <- FindNeighbors(object = tcells, dims = 1:18)

In [ ]:
tcells <- FindClusters(object = tcells, resolution = 0.8)

## Run Non-linear dimensional reduction (tSNE) and UMAP

In [ ]:
tcells

In [ ]:
tcells <- RunUMAP(tcells, dims = 1:18)

In [ ]:
#umap, dims 18, res. 0.7
DimPlot(tcells, reduction = 'umap', label = TRUE)
DimPlot(tcells, reduction = 'umap', label = TRUE, group.by = 'orig.ident')
DimPlot(tcells, reduction = 'umap', label = TRUE, group.by = 'TCR_frequency')

# Cluster markers

In [ ]:
DefaultAssay(tcells) <- 'RNA'

In [ ]:
#run the DGE for each cluster
featurestcells <- rownames(tcells)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS", x = rownames(tcells), value = TRUE)
featurestcells <- featurestcells[!(featurestcells%in%markers.remove)]
tcells.markers1 <- FindAllMarkers(object = tcells, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = featurestcells)

In [ ]:
#sort markers, save
tcells.markers1_sorted <- c()
for (i in 1:length(levels(tcells.markers1$cluster))){
    tcells.markers1_level <- filter(tcells.markers1, cluster == levels(tcells.markers1$cluster)[i])
    tcells.markers1_level <- tcells.markers1_level[order(-tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_level <- tcells.markers1_level[1:30, ]
    tcells.markers1_level <- tcells.markers1_level[!is.na(tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_sorted <- rbind(tcells.markers1_sorted, tcells.markers1_level)
    }
tcells.markers1_sorted_top30 <- tcells.markers1_sorted
tcells.markers1_sorted_top30
write.csv(tcells.markers1_sorted_top30, file = './tcells_first_markers_top30.csv')

In [ ]:
#Create average expression object
cluster.averages_tcells <- AverageExpression(tcells, assay = "RNA", return.seurat = TRUE) # , verbose = FALSE)

In [ ]:
#Sort markers, plot
tcells.markers1_sorted <- c()
for (i in 1:length(levels(tcells.markers1$cluster))){
    tcells.markers1_level <- filter(tcells.markers1, cluster == levels(tcells.markers1$cluster)[i])
    tcells.markers1_level <- tcells.markers1_level[order(-tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_level <- tcells.markers1_level[1:5, ]
    tcells.markers1_level <- tcells.markers1_level[!is.na(tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_sorted <- rbind(tcells.markers1_sorted, tcells.markers1_level)
    }
tcells.markers1_sorted_top5 <- tcells.markers1_sorted
tcells.markers1_sorted_top5

In [ ]:
#look at the number of features per cell in the clusters
VlnPlot(tcells, features = 'nFeature_RNA', pt.size = 0.01)

# Exploring the markers, defining the clusters

In [ ]:
levels(tcells)

In [ ]:
# Print all clusters
cluster_to_plot <- 'here the number of cluster'
        print(DotPlot(tcells, features = filter(tcells.markers1_sorted_top30, cluster == cluster_to_plot)$gene, dot.scale = 8) + RotatedAxis() +
                theme(axis.line = element_line(size=1),
                text = element_text(size = 17),
                axis.text = element_text(size = 17),
                axis.ticks = element_line(size=1),
                legend.text=element_text(size=17)) + ggtitle(cluster_to_plot))
       print(cluster_to_plot)
        
        for(i in 1:nrow(filter(tcells.markers1_sorted_top30, cluster == cluster_to_plot))){
             gene_to_plot <- filter(tcells.markers1_sorted_top30, cluster == cluster_to_plot)$gene[i]
                print(VlnPlot(tcells, features = gene_to_plot, pt.size = 0.01) +
                theme(axis.line = element_line(size=1),
                      text = element_text(size = 20),
                      axis.text = element_text(size = 20),
                      axis.ticks = element_line(size=1),
                      legend.text=element_text(size=20)) + ggtitle(paste(cluster_to_plot, ' ', gene_to_plot, sep = '')))
                print(gene_to_plot)
                print(FeaturePlot(tcells, features = gene_to_plot, label.size = 8, label = TRUE, pt.size = 1) + 
                theme(axis.line = element_line(size=1),
                      text = element_text(size = 20),
                      axis.text = element_text(size = 20),
                      axis.ticks = element_line(size=1),
                      legend.text=element_text(size=20))+ ggtitle(paste(cluster_to_plot, ' ', gene_to_plot, sep = '')))
        }       

# Cell cycle scoring

In [ ]:
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes
tcells <- CellCycleScoring(tcells, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)
head(tcells@meta.data)

In [ ]:
DimPlot(tcells, reduction = 'umap', group.by = 'Phase', label = TRUE, pt.size = 0.5, label.size = 6)

In [ ]:
Idents(tcells) <- 'seurat_clusters'
levels(tcells)

# Rename the clusters

In [ ]:
tcells@meta.data$cd8names <- 'FALSE'

In [ ]:
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 8] <- '1_CCR7'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 3] <- '2_MAL'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 12] <- '3_NELL2'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 0] <- '4_CMC1'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 2] <- '5_FGFBP2'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 6] <- '6_FCGR3A'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 4] <- '7_LAG3'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 5] <- '8_MKI67'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 1] <- '8_MKI67'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 7] <- '9_NME1'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 10] <- '9_NME1'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 9] <- 'MAIT_1'
tcells@meta.data$cd8names[tcells@meta.data$seurat_clusters == 11] <- 'MAIT_2'
unique(tcells@meta.data$cd8names)

In [ ]:
Idents(tcells) <- 'cd8names'
levels(tcells) <- c('1_CCR7', '2_MAL', '3_NELL2', '4_CMC1', '5_FGFBP2', '6_FCGR3A', '7_LAG3', '8_MKI67', '9_NME1', 'MAIT_1', 'MAIT_2')

# Adding expanded column

In [ ]:
#add expand column
for (i in 1:nrow(tcells@meta.data)){
    if(tcells@meta.data$TCR_frequency[i] > 2){
        tcells@meta.data$expand[i] <- 'exp'
    } else {tcells@meta.data$expand[i] <- 'nonexp'}
}

In [ ]:
DimPlot(tcells, reduction = 'umap', group.by = 'TCR_frequency', label = TRUE)
DimPlot(tcells, reduction = 'umap', group.by = 'expand')

In [ ]:
pbmc <- tcells

# Subset out the cells with more then one beta and alpha chain

In [ ]:
#let's find t cell doublets:
for(i in 1:nrow(pbmc@meta.data)){
         pbmc@meta.data$tcr_b_sum[i] <- sum(pbmc@meta.data[i, c('TCR1B', 'TCR2B', 'TCR3B', 'TCR4B')] != 'FALSE')
    }

In [ ]:
#save the t doublets: 
write.csv(filter(pbmc@meta.data, tcr_b_sum > 1), file = './cd8_tcelldoublets.csv')
#save cells where only alfa chain was found:
write.csv(filter(pbmc@meta.data, tcr_b_sum == 0), file = './cd8_tcell_no_betachain.csv')
tcellstoremove <- c(rownames(filter(pbmc@meta.data, tcr_b_sum > 1)))
length(tcellstoremove)

In [ ]:
#let's find t cell doublets:
for(i in 1:nrow(pbmc@meta.data)){
         pbmc@meta.data$tcr_a_sum[i] <- sum(pbmc@meta.data[i, c('TCR1A', 'TCR2A', 'TCR3A', 'TCR4A')] != 'FALSE')
    }

In [ ]:
#save the t doublets: 
write.csv(filter(pbmc@meta.data, tcr_a_sum > 1), file = './cd8_tcelldoublets_alpha.csv')
#save cells where only alfa chain was found:
write.csv(filter(pbmc@meta.data, tcr_a_sum == 0), file = './cd8_tcell_no_betachain_alpa.csv')
tcellstoremove <- c(rownames(filter(pbmc@meta.data, tcr_a_sum > 1)), tcellstoremove)
length(tcellstoremove)

In [ ]:
#subset everything without doublets or cells without beta chain:
pbmc <- subset(pbmc, cells = tcellstoremove, invert = TRUE)
pbmc

# Assign the clonotypes

In [ ]:
clonotypes <- read.csv2('~/clonotype_info.csv')

In [ ]:
pbmc@meta.data$clonotypes <- 'FALSE'
pbmc@meta.data$specificity <- 'FALSE'

#First check TCR beta
for(i in 1:nrow(pbmc@meta.data)){
    if(pbmc@meta.data$TCR1B[i] %in% clonotypes$CDR3.beta){
      pbmc@meta.data$clonotypes[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR1B[i])$TCR  
      pbmc@meta.data$specificity[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR1B[i])$Specificity  
    }
    
    if(pbmc@meta.data$TCR2B[i] %in% clonotypes$CDR3.beta){
      pbmc@meta.data$clonotypes[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR2B[i])$TCR  
      pbmc@meta.data$specificity[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR2B[i])$Specificity  
    }
    
    if(pbmc@meta.data$TCR3B[i] %in% clonotypes$CDR3.beta){
      pbmc@meta.data$clonotypes[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR3B[i])$TCR  
      pbmc@meta.data$specificity[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR3B[i])$Specificity  
    }
    
    if(pbmc@meta.data$TCR4B[i] %in% clonotypes$CDR3.beta){
      pbmc@meta.data$clonotypes[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR4B[i])$TCR  
      pbmc@meta.data$specificity[i] <- filter(clonotypes, CDR3.beta == pbmc@meta.data$TCR4B[i])$Specificity  
    }
}

In [ ]:
#now let's check the alpha
pbmc@meta.data$clonotypes_2 <- 'FALSE'
for(i in 1:nrow(pbmc@meta.data)){
    if(pbmc@meta.data$clonotypes[i] != 'FALSE'){
        if((pbmc@meta.data$TCR1A[i] %in% clonotypes$CDR3.alpha) & (pbmc@meta.data$clonotypes[i] == filter(clonotypes, CDR3.alpha == pbmc@meta.data$TCR1A[i])$TCR[1])){
        pbmc@meta.data$clonotypes_2[i] <- 'TRUE'
        } 
        if((pbmc@meta.data$TCR2A[i] %in% clonotypes$CDR3.alpha) & (pbmc@meta.data$clonotypes[i] == filter(clonotypes, CDR3.alpha == pbmc@meta.data$TCR2A[i])$TCR[1])){
        pbmc@meta.data$clonotypes_2[i] <- 'TRUE'
        } 
        if((pbmc@meta.data$TCR3A[i] %in% clonotypes$CDR3.alpha) & (pbmc@meta.data$clonotypes[i] == filter(clonotypes, CDR3.alpha == pbmc@meta.data$TCR3A[i])$TCR[1])){
        pbmc@meta.data$clonotypes_2[i] <- 'TRUE'
        } 
        if((pbmc@meta.data$TCR4A[i] %in% clonotypes$CDR3.alpha) & (pbmc@meta.data$clonotypes[i] == filter(clonotypes, CDR3.alpha == pbmc@meta.data$TCR4A[i])$TCR[1])){
        pbmc@meta.data$clonotypes_2[i] <- 'TRUE'
        } 
    }
}

In [ ]:
#now correct the clonotypes column
for(i in 1:nrow(pbmc@meta.data)){
    if(pbmc@meta.data$clonotypes_2[i] == 'FALSE'){
        pbmc@meta.data$clonotypes[i] <- 'FALSE'
        pbmc@meta.data$specificity[i] <- 'FALSE'
    }
}
pbmc@meta.data$clonotypes_2 <- NULL

In [ ]:
#combine the H1 and H2 with the clonotypes info
pbmc@meta.data$s_clonotypes <- 'FALSE'
for(i in 1:nrow(pbmc@meta.data)){
    if(pbmc@meta.data$clonotypes[i] != 'FALSE'){
    pbmc@meta.data$s_clonotypes[i] <- paste0('H', gsub(x = pbmc@meta.data$orig.ident[i], pattern = '.*_', replacement = ''), '_', pbmc@meta.data$clonotypes[i])
    }
}
unique(pbmc@meta.data$s_clonotypes)

In [ ]:
# unique clonotypes per sample

pbmc@meta.data$sample_clono <- paste0(pbmc@meta.data$orig.ident, '_', pbmc@meta.data$TCR_clonotype_id)
pbmc@meta.data$TCR_frequency_corrected <- 'FALSE'

for(i in 1:nrow(pbmc@meta.data)){
    pbmc@meta.data$TCR_frequency_corrected[i] <- nrow(filter(pbmc@meta.data, sample_clono == pbmc@meta.data$sample_clono[i]))
}

df <- pbmc@meta.data
df <- df[!duplicated(df$sample_clono), ]
rownames(df) <- 1:nrow(df)
df <- df[!(colnames(df) %in% c('nCount_RNA', 'nFeature_RNA', 'percent.mito', 'integrated_snn_res.0.8', 'seurat_clusters', 'integrated_snn_res.0.7', 'integrated_snn_res.0.6', 
           'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'cd8names', 'tcr_b_sum', 'tcr_a_sum'))]
nrow(df)
write.csv(df, file = 'unique_clonotypes_20221127.csv')

In [ ]:
#combine orig ident and speicificity column
pbmc@meta.data$ident_specificity <- 'FALSE'
for(i in 1:nrow(pbmc@meta.data)){
    if(pbmc@meta.data$orig.ident[i] == 'pbmc_1'){pbmc@meta.data$ident_specificity[i] <- paste0('H1', '_', pbmc@meta.data$specificity[i])}
    if(pbmc@meta.data$orig.ident[i] == 'pbmc_2'){pbmc@meta.data$ident_specificity[i] <- paste0('H2', '_', pbmc@meta.data$specificity[i])}
}
pbmc@meta.data$ident_specificity[pbmc@meta.data$ident_specificity == 'H1_FALSE'] <- 'H1_Other'
pbmc@meta.data$ident_specificity[pbmc@meta.data$ident_specificity == 'H2_FALSE'] <- 'H2_Other'

# Plotting

In [ ]:
dir.create('outs_plots')

In [ ]:
#for the UMAP clusters
options(repr.plot.width = 14, repr.plot.height = 11)
plot <- DimPlot(pbmc, reduction = "umap", label = TRUE, label.size = 8, label.box = TRUE, label.color = 'black', repel = TRUE, pt.size = 0.8, cols = brewer.pal(length(levels(pbmc)), name = 'Paired')) + 
theme(
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))
plot
options(repr.plot.width = 11, repr.plot.height = 11)
ggsave(plot, file = './outs_plots/UMAP_allclusters_withlabel.eps', width = 14, height = 11)
ggsave(plot, file = './outs_plots/UMAP_allclusters_withlabel.pdf', width = 14, height = 11)

In [ ]:
#stim vs unstim
width_p <- 13
height_p <- 11
dir <- './outs_plots/'
plot_name <- 'stim_vs_unstim_umap'

pbmc@meta.data$stimulation <- 'FALSE'
pbmc@meta.data$stimulation[pbmc@meta.data$orig.ident == 'pbmc_2'] <- 'unstimulated'
pbmc@meta.data$stimulation[pbmc@meta.data$orig.ident == 'pbmc_1'] <- 'stimulated'


options(repr.plot.width = width_p, repr.plot.height = height_p)
plot <- DimPlot(pbmc, reduction = "umap", label = FALSE, label.size = 10, repel = TRUE, group.by = 'stimulation', cols = c("#D3556E", 'lightgrey'), pt.size = 0.8) + 
theme(
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))

print(plot)
ggsave(plot, file = paste0(dir, plot_name, '.pdf'), width = width_p, height = height_p)
ggsave(plot, file = paste0(dir, plot_name, '.eps'), width = width_p, height = height_p)
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
#UMAP with cell stages
options(repr.plot.width = 13, repr.plot.height = 11)
plot <- DimPlot(pbmc, reduction = "umap", label = TRUE, label.size = 10, repel = TRUE, group.by = 'Phase', pt.size = 0.8) + 
theme(
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))
plot
ggsave(plot, file = './outs_plots/Cell_Phase_regression.eps', width = 13, height = 11)
ggsave(plot, file = './outs_plots/Cell_Phase_regression.pdf', width = 13, height = 11)
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
#plot the markers
featurespbmc <- rownames(pbmc)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^IGKV|^IGLV|^IGHV|^IGHG", x = rownames(pbmc), value = TRUE)
featurespbmc <- featurespbmc[!(featurespbmc%in%markers.remove)]
pbmc.markers1 <- FindAllMarkers(object = pbmc, only.pos = TRUE, min.pct = 0.2, logfc.threshold = 0.2, features = featurespbmc)
write.csv(pbmc.markers1, file = 'markers_unsorted.csv')

In [ ]:
#sort markers for plotting
pbmc.markers1_sorted <- c()
for (i in 1:length(levels(pbmc.markers1$cluster))){
    pbmc.markers1_level <- filter(pbmc.markers1, cluster == levels(pbmc.markers1$cluster)[i])
    pbmc.markers1_level <- pbmc.markers1_level[order(-pbmc.markers1_level$avg_log2FC), ]
    pbmc.markers1_level <- pbmc.markers1_level[pbmc.markers1_level$p_val_adj < 0.05, ]
    pbmc.markers1_level <- pbmc.markers1_level[1:5, ]
    pbmc.markers1_level <- pbmc.markers1_level[!is.na(pbmc.markers1_level$avg_log2FC), ]
    pbmc.markers1_sorted <- rbind(pbmc.markers1_sorted, pbmc.markers1_level)
    }
pbmc.markers1_sorted_top5 <- pbmc.markers1_sorted

In [ ]:
object <- pbmc
levels(object) <- rev(levels(pbmc))
cluster.averages_pbmc <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

In [ ]:
#build the heatmap
options(repr.plot.width = 8, repr.plot.height = 22)
hm <- DoHeatmap(cluster.averages_pbmc, features = pbmc.markers1_sorted_top5$gene, group.colors = rev(brewer.pal(length(levels(pbmc)), name = 'Paired')), draw.lines = FALSE,size = 8, angle = 270, hjust = 1, raster = FALSE) + theme(text = element_text(size = 12, face = "bold")) + 
scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E")) + theme(text = element_text(size = 25))
ggsave(hm, file = './outs_plots/heatmap_clusters.eps', width = 8, height = 22)
ggsave(hm, file = './outs_plots/heatmap_clusters.pdf', width = 8, height = 22)
hm
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
#build the dotplot
options(repr.plot.width = 20, repr.plot.height = 8)
object <- pbmc
levels(object) <- rev(levels(object))
plot <- DotPlot(object, features = unique(pbmc.markers1_sorted_top5$gene), dot.scale = 10, cols = c('white', '#D3556E')) + RotatedAxis() +
        theme(
        text = element_text(size = 17),
        axis.text = element_text(size = 17),
        legend.text=element_text(size=17))
plot
ggsave(plot, file = './outs_plots/featuremap_clusters.eps', width = 20, height = 8)
ggsave(plot, file = './outs_plots/featuremap_clusters.pdf', width = 20, height = 8)
options(repr.plot.width = 11, repr.plot.height = 11)

# Plot the markers

In [ ]:
object <- pbmc
object_average <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

In [ ]:
#featureplot
markers <- c('CCR7', 'LEF1', 'NELL2', 'MAL', 'MYC', 'SELL', 'TCF7', 'NOSIP')

markers <- list(markers)
object <- AddModuleScore(object, features = markers, name = 'markers')
plot <- FeaturePlot(object, features = 'markers1', label.size = 6, label = TRUE, pt.size = 1, repel = TRUE, cols = c("lightgrey", "#D3556E"), min.cutoff = 'q15') + 
                theme(axis.title = element_text(size = 20),
                        plot.title = element_text(size = 20, vjust = -5, face = 'plain'),
                      axis.text = element_text(size = 20),
                      legend.text=element_text(size=20))+
                 ggtitle(paste(markers[[1]], collapse = ', '))
plot
ggsave(plot, file = './outs_plots/naiive_signature.eps', width = 11, height = 11)
ggsave(plot, file = './outs_plots/naiive_signature.pdf', width = 11, height = 11)

In [ ]:
#featureplot
markers2 <- markers
markers <- c('CX3CR1', 'GNLY', 'GZMH', 'FGFBP2', 'FCGR3A', 'PLEK', 'ADGRG1', 'PRF1')

markers <- list(markers)
object <- AddModuleScore(object, features = markers, name = 'markers')

plot <- FeaturePlot(object, features = 'markers1', label.size = 6, label = TRUE, pt.size = 1, repel = TRUE, cols = c("lightgrey", "#D3556E"), min.cutoff = 'q15') + 
                theme(axis.title = element_text(size = 20),
                        plot.title = element_text(size = 20, vjust = -5, face = 'plain'),
                      axis.text = element_text(size = 20),
                      legend.text=element_text(size=20))+
                 ggtitle(paste(markers[[1]], collapse = ', '))
plot
ggsave(plot, file = './outs_plots/effector_signature.eps', width = 11, height = 11)
ggsave(plot, file = './outs_plots/effector_signature.pdf', width = 11, height = 11)

In [ ]:
#heatmap
combined <- c(markers2[[1]], markers[[1]])
options(repr.plot.width = 8, repr.plot.height = 11)
plot <- DoHeatmap(object_average, features = combined, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = brewer.pal(length(levels(pbmc)), name = 'Paired')) + theme(text = element_text(size = 20, face = "bold"))  + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
plot
ggsave(plot, file = './outs_plots/hm_naiive_effector.eps', width = 8, height = 11)
ggsave(plot, file = './outs_plots/hm_naiive_effector.pdf', width = 8, height = 11)
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
#featureplot
markers <- c('LAG3', 'TIGIT', 'HAVCR2')

markers <- list(markers)
object <- AddModuleScore(object, features = markers, name = 'markers')
plot <- FeaturePlot(object, features = 'markers1', label.size = 6, label = TRUE, pt.size = 1, repel = TRUE, cols = c("lightgrey", "#D3556E"), min.cutoff = 'q15') + 
                theme(axis.title = element_text(size = 20),
                        plot.title = element_text(size = 20, vjust = -5, face = 'plain'),
                      axis.text = element_text(size = 20),
                      legend.text=element_text(size=20))+
                 ggtitle(paste(markers[[1]], collapse = ', '))
plot
ggsave(plot, file = './outs_plots/negative_signature.eps', width = 11, height = 11)
ggsave(plot, file = './outs_plots/negative_signature.pdf', width = 11, height = 11)

In [ ]:
#featureplot
markers2 <- markers
markers <- c('MKI67', 'HIST1H4C', 'HSPD1', 'NME1', 'HSP90AB1', 'ENO1', 'EIF4A1')

markers <- list(markers)
object <- AddModuleScore(object, features = markers, name = 'markers')

plot <- FeaturePlot(object, features = 'markers1', label.size = 6, label = TRUE, pt.size = 1, repel = TRUE, cols = c("lightgrey", "#D3556E"), min.cutoff = 'q15') + 
                theme(axis.title = element_text(size = 20),
                        plot.title = element_text(size = 20, vjust = -5, face = 'plain'),
                      axis.text = element_text(size = 20),
                      legend.text=element_text(size=20))+
                 ggtitle(paste(markers[[1]], collapse = ', '))
plot
ggsave(plot, file = './outs_plots/proliferative_signature.eps', width = 11, height = 11)
ggsave(plot, file = './outs_plots/proliferative_signature.pdf', width = 11, height = 11)

In [ ]:
#heatmap
combined <- c(markers2[[1]], markers[[1]])
width <- 8
height <- 6.5
options(repr.plot.width = width, repr.plot.height = height)
plot <- DoHeatmap(object_average, features = combined, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = brewer.pal(length(levels(pbmc)), name = 'Paired')) + theme(text = element_text(size = 20, face = "bold"))  + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
plot
ggsave(plot, file = './outs_plots/hm_negative_proliferative.eps', width = width, height = height)
ggsave(plot, file = './outs_plots/hm_negative_proliferative.pdf', width = width, height = height)

options(repr.plot.width = 11, repr.plot.height = 11)

# Pathway enrichment analysis per cluster

In [ ]:
library(httr)
httr::set_config(config(ssl_verifypeer = FALSE))
library('enrichR')
setEnrichrSite("Enrichr")
dir.create('outs_clusters')

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('1_CCR7', '2_MAL', '3_NELL2', '4_CMC1', '5_FGFBP2', '6_FCGR3A', '7_LAG3', '8_MKI67', '9_NME1', 'MAIT_1', 'MAIT_2')
group2 <- NULL
object <- pbmc
grouping <- 'cd8names'
grouping_levels <- c('1_CCR7', '2_MAL', '3_NELL2', '4_CMC1', '5_FGFBP2', '6_FCGR3A', '7_LAG3', '8_MKI67', '9_NME1', 'MAIT_1', 'MAIT_2')
Idents(object) <- grouping
levels(object) <- grouping_levels
cutoff_expression <- 0.2 #for the DGE min.pct
logfc_cutoff <- 0.2 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC|^MRPL", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]
#run the DGE for all clusters
pbmc.markers <- FindAllMarkers(object = pbmc, only.pos = TRUE, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff, features = features)

In [ ]:
pbmc.markers$cluster <- as.character(pbmc.markers$cluster)
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('./outs_clusters/', cluster_of_interest[cl])
    dir.create(dir_path)
    
    #subset the genes of interest 
    de_genes <- filter(pbmc.markers, cluster == cluster_of_interest[cl])
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_positive_', '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = '#d3556e') +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl],  'positive', databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = '#d3556e') +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl],  'positive', databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_positive', '.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE) + theme(text = element_text(size = 20, face = "bold"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_HEATMAP_genes_positive', '.pdf'), width = 8, height = 11)
        
        vlnplot <- VlnPlot(object, features = genes_to_plot, stack = TRUE, flip = TRUE) +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))
        ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_StackedVIOLIN_genes_positive', '.pdf'), width = 8, height = 11)
        }
    #end of the einrichr loop
}
#end of the clusters loop

# Expansion overview

In [ ]:
options(repr.plot.width=15, repr.plot.height=13)

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

df <- pbmc@meta.data
df$clusters <- pbmc$cd8names
clusters <- unique(df$clusters)
dis_state <- 'exp'
df$dis <- df$expand
df$dis <- factor(df$dis, levels = c('nonexp', 'exp'))

#determine where are the most cells from HL
order_df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters)))
colnames(order_df) <- c('cluster', 'dis')
order_df$cluster <- clusters
for(i in 1:nrow(order_df)){
    order_df$dis[i] <- 100 * nrow(filter(df, dis == dis_state & clusters == order_df$cluster[i])) / nrow(filter(df, clusters == order_df$cluster[i]))
}

order_df <- order_df[order(order_df$dis), ]
order_list <- order_df$cluster

#order the clusters in the df 
df$clusters <- factor(df$clusters, levels = order_list)

#plot the type of disease state
plot <- ggplot(df, aes_string(x="clusters", fill="dis")) +
    geom_bar(position="fill") + 
    theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=25),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"),
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1, colour = 'black')) +
        scale_fill_manual('legend', values = c('lightgrey', '#D3556E'))+ ylab('Fraction')+
    ggtitle("Expanded cells fraction")
plot
ggsave(plot, file = './outs_plots/expanded_cells_fraction.pdf', width = 12, height = 8)
ggsave(plot, file = './outs_plots/expanded_cells_fraction.eps', width = 12, height = 8)

In [ ]:
# Numbers of expanded and non-expanded cells:
expvsnonexp <- data.frame(matrix(NA, ncol = length(unique(pbmc@meta.data$cd8names)), nrow = 5))
colnames(expvsnonexp) <- unique(pbmc@meta.data$cd8names)
rownames(expvsnonexp) <- c('expanded abs', 'exp relative', 'non-expanded abs', 'non-exp relative', 'total')
for (i in 1:length(unique(pbmc@meta.data$cd8names))){
    exp <- nrow(filter(pbmc@meta.data, expand == 'exp' & cd8names == unique(pbmc@meta.data$cd8names)[i]))
    exprel <- 100*nrow(filter(pbmc@meta.data, expand == 'exp' & cd8names == unique(pbmc@meta.data$cd8names)[i])) / nrow(filter(pbmc@meta.data, cd8names == unique(pbmc@meta.data$cd8names)[i]))
    nonexp <- nrow(filter(pbmc@meta.data, expand == 'nonexp' & cd8names == unique(pbmc@meta.data$cd8names)[i]))
    nonexprel <- 100*nrow(filter(pbmc@meta.data, expand == 'nonexp' & cd8names == unique(pbmc@meta.data$cd8names)[i])) / nrow(filter(pbmc@meta.data, cd8names == unique(pbmc@meta.data$cd8names)[i]))
    total <- nrow(filter(pbmc@meta.data, cd8names == unique(pbmc@meta.data$cd8names)[i]))
    expvsnonexp[1, i] <- exp
    expvsnonexp[2, i] <- exprel
    expvsnonexp[3, i] <- nonexp
    expvsnonexp[4, i] <- nonexprel
    expvsnonexp[5, i] <- total            
}        
#order in increasing order according to expansion
expvsnonexp1 <- expvsnonexp[, order(expvsnonexp[2, ])]
expvsnonexp1
write.csv(expvsnonexp1, file = './cd8expansion.csv')

# Explore in which clusters are the clonotypes

In [ ]:
library(RColorBrewer)
nb.cols <- 15
mycolors <- colorRampPalette(brewer.pal(8, "Paired"))(nb.cols)

In [ ]:
pbmc@meta.data$s_clonotypes_2 <- pbmc@meta.data$s_clonotypes
pbmc@meta.data$s_clonotypes[pbmc@meta.data$s_clonotypes == 'H2_4JU'] <- 'FALSE'
pbmc@meta.data$s_clonotypes[pbmc@meta.data$s_clonotypes == 'H1_4JU'] <- 'FALSE'
unique(pbmc@meta.data$s_clonotypes)

In [ ]:
#sytl4_til1 #9e1c20
#sytl4_til2 #f36a2a
#sytl4_pbc1 #f89d4e
#sytl4_pbc2 #f59b9b

#kif2c_pbc1 #475fad
#kif2c_pbc2 #85b6e1

#ju1 #6d08e6
#ju2 #9568bc


pbmc@meta.data$s_clonotypes[pbmc@meta.data$s_clonotypes == 'FALSE'] <- 'Non-assigned'
Idents(pbmc) <- 's_clonotypes'
levels(pbmc) <- c('H1_KIF2C-PBC1', 'H1_KIF2C-PBC2', 'H1_SYTL4-PBC1', 'H1_SYTL4-PBC2', 'H1_SYTL4-TIL1', 'H1_SYTL4-TIL2', 'H1_JU1', 'H1_JU2', 'H2_KIF2C-PBC1', 'H2_KIF2C-PBC2', 'H2_SYTL4-PBC1', 'H2_SYTL4-PBC2', 'H2_JU1', 'Non-assigned')
colours <- c('#5cadfc', '#002ef2', '#ffa140', '#ffe0bf', '#950202', '#ff6100', '#45f000', '#1d9d00', '#5cadfc', '#002ef2', '#ffa140', '#ffe0bf', '#45f000', 'grey')
numbers <- data.frame(matrix(NA, ncol = 2, nrow = length(levels(pbmc))))
colnames(numbers) <- c('clonotype', 'abs_number')
numbers$clonotype <- levels(pbmc)
for(i in 1:nrow(numbers)){
    numbers$abs_number[i] <- nrow(filter(pbmc@meta.data, s_clonotypes == numbers$clonotype[i]))
}

options(repr.plot.width = 14, repr.plot.height = 11)
plot <- DimPlot(pbmc, reduction = "umap", label = TRUE, label.size = 7, repel = TRUE, pt.size = 0.8, cols = colours, label.box = TRUE, label.color = 'white')+ 
theme(
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20)) + scale_color_manual(values = colours,
                     labels = paste0(levels(pbmc), ' (', numbers$abs_number, ')'))
plot 
ggsave(plot, width = 14, height = 11, file = './outs_plots/clonotypes_all_onumap.eps')
ggsave(plot, width = 14, height = 11, file = './outs_plots/clonotypes_all_onumap.pdf')
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
s_clonotypes <- c('H1_KIF2C-PBC1', 'H1_KIF2C-PBC2', 'H1_SYTL4-PBC1', 'H1_SYTL4-PBC2', 'H1_SYTL4-TIL1', 'H1_SYTL4-TIL2', 'H1_JU1', 'H1_JU2', 'H2_KIF2C-PBC1', 'H2_KIF2C-PBC2', 'H2_SYTL4-PBC1', 'H2_SYTL4-PBC2', 'H2_JU1')
clusters <- levels(pbmc)
phases <- unique(pbmc@meta.data$Phase)

columns <- levels(pbmc)
object <- pbmc


#build up a dataframe
df <- data.frame(matrix(NA, ncol = length(columns), nrow = length(s_clonotypes)))
colnames(df) <- columns
rownames(df) <- s_clonotypes
for(i in 1:nrow(df)){
    for(c in 1:ncol(df)){
        df[i, c] <- 100*nrow(filter(object@meta.data, s_clonotypes == rownames(df)[i] & cd8names == colnames(df)[c])) / nrow(filter(object@meta.data, s_clonotypes == rownames(df)[i]))
    }
}
#aggregate 
aggregate <- c()
for(i in 1:nrow(df)){
    for(c in 1:ncol(df)){
        result <- data.frame(rownames(df)[i], colnames(df)[c], df[i, c])
        colnames(result) <- c('clonotype', 'column', 'result')
        aggregate <- rbind(aggregate, result)
    }
}

aggregate$clonotype <- factor(aggregate$clonotype, levels = s_clonotypes)

In [ ]:
options(repr.plot.width=14, repr.plot.height=11)
plot <- ggplot(aggregate, aes(fill=column, y=result, x=clonotype)) + 
        geom_bar(position="stack", stat="identity", ) + theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=30),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"), 
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1)) + ylab('Fraction within the cluster')+ xlab('Clonotype')+
        #scale_fill_manual(cbPalette)
        #scale_fill_viridis(discrete = TRUE) 
        scale_fill_brewer(palette = "Paired")
    print(plot)
    ggsave(plot, width = 14, height = 11, file = './outs_plots/fraction_within_cluster.eps')
    ggsave(plot, width = 14, height = 11, file = './outs_plots/fraction_within_cluster.pdf')
options(repr.plot.width=11, repr.plot.height=11)

In [ ]:
colonotypes_list <- list(c('H1_KIF2C-PBC1', 'H2_KIF2C-PBC1'), 
                         c('H1_KIF2C-PBC2', 'H2_KIF2C-PBC2'),
                        c('H1_SYTL4-PBC1', 'H2_SYTL4-PBC1'),
                        c('H1_SYTL4-PBC2', 'H2_SYTL4-PBC2'),
                        c('H1_SYTL4-TIL1'),
                        c('H1_SYTL4-TIL2'),
                        c('H1_JU1', 'H2_JU1'),
                        c('H1_JU2')
                        )
colonotypes_list

In [ ]:
#stim vs unstim UMAPs for each clonotype
width_p <- 13
height_p <- 11
dir <- './outs_plots/single_clonotypes_new/'
dir.create(dir)

options(repr.plot.width = width_p, repr.plot.height = height_p)

clonotypes_list <- list(c('H1_KIF2C-PBC1', 'H2_KIF2C-PBC1'), 
                         c('H1_KIF2C-PBC2', 'H2_KIF2C-PBC2'),
                        c('H1_SYTL4-PBC1', 'H2_SYTL4-PBC1'),
                        c('H1_SYTL4-PBC2', 'H2_SYTL4-PBC2'),
                        c('H1_SYTL4-TIL1'),
                        c('H1_SYTL4-TIL2'),
                        c('H1_JU1', 'H2_JU1'),
                        c('H1_JU2')
                        )


for(i in 1:length(clonotypes_list)){
    clonotypes_to_plot <- clonotypes_list[[i]]
        
    plot_name <- paste0(clonotypes_to_plot[1], '_stimunstim.eps')
    
    pbmc@meta.data$clono_plot <- 'unassigned'
    
    if(length(clonotypes_to_plot) > 1){
    pbmc@meta.data$clono_plot[pbmc@meta.data$s_clonotypes == clonotypes_to_plot[1]] <- clonotypes_to_plot[1]
    pbmc@meta.data$clono_plot[pbmc@meta.data$s_clonotypes == clonotypes_to_plot[2]] <- clonotypes_to_plot[2]
    colors <- c('lightgrey', "#D3556E", '#475fad')
    pbmc$clono_plot <- factor(pbmc$clono_plot, levels = c('unassigned', clonotypes_to_plot))
    
    }
        
    if(length(clonotypes_to_plot) == 1){
    pbmc@meta.data$clono_plot[pbmc@meta.data$s_clonotypes == clonotypes_to_plot[1]] <- clonotypes_to_plot[1]
    colors <- c('lightgrey', "#D3556E")
    pbmc$clono_plot <- factor(pbmc$clono_plot, levels = c('unassigned', clonotypes_to_plot))
    }
    
    plot <- DimPlot(pbmc, reduction = "umap", label = FALSE, order = TRUE, label.size = 10, repel = TRUE, group.by = 'clono_plot', cols = colors, pt.size = 2) + 
    theme(
          text = element_text(size = 20),
          plot.title = element_text(size = 20),
          axis.text = element_text(size = 20),
          legend.text=element_text(size=20)) + ggtitle(clonotypes_to_plot[1])
    
    print(plot)
    ggsave(plot, file = paste0(dir, plot_name), width = width_p, height = height_p)
        
    plot <- DimPlot(pbmc, reduction = "umap", label = FALSE, order = TRUE, label.size = 10, repel = TRUE, group.by = 'clono_plot', cols = colors, pt.size = 2) + 
    theme(
          text = element_text(size = 20),
          plot.title = element_text(size = 20),
          axis.text = element_text(size = 20),
          legend.text=element_text(size=20)) + ggtitle(clonotypes_to_plot[1]) + NoLegend()
    ggsave(plot, file = paste0(dir, 'Nolegend', plot_name), width = width_p, height = height_p)
    
}
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
#stim vs unstim
width_p <- 13
height_p <- 11
dir <- './outs_plots/'
plot_name <- 'assigned_vs_nonassigned'

pbmc@meta.data$assigned <- 'non-assigned'
pbmc@meta.data$assigned[pbmc@meta.data$clonotypes != 'FALSE'] <- 'assigned'


options(repr.plot.width = width_p, repr.plot.height = height_p)
plot <- DimPlot(pbmc, reduction = "umap", label = FALSE, label.size = 10, repel = TRUE, group.by = 'assigned', cols = c("#D3556E", 'lightgrey'), pt.size = 0.8) + 
theme(
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))

print(plot)
ggsave(plot, file = paste0(dir, plot_name, '.pdf'), width = width_p, height = height_p)
ggsave(plot, file = paste0(dir, plot_name, '.eps'), width = width_p, height = height_p)
options(repr.plot.width = 11, repr.plot.height = 11)

# DGE between the clonotypes

In [ ]:
library(httr)
httr::set_config(config(ssl_verifypeer = FALSE))
library('enrichR')
setEnrichrSite("Enrichr")
dir.create('outs')

In [ ]:
#create grouping 

pbmc@meta.data$h1_general <- 'FALSE'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'

## JU vs SYTL4

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('SYTL4')
group2 <- 'JU'
object <- pbmc
grouping <- 'h1_general'
grouping_levels <- c('KIF2C', 'JU', 'SYTL4', 'FALSE')
colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#bebebe')
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]
Idents(object) <- grouping
levels(object) <- grouping_levels
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#for the heatmap
object_wf <- subset(object, idents = 'FALSE', invert = TRUE)
levels(object_wf) <- c('KIF2C', 'JU', 'SYTL4')
object_av_wf <- AverageExpression(object_wf, assay = "RNA", return.seurat = TRUE, verbose = FALSE)
colours_diagnosis_groups_wf <- c('#5cadfb', '#1d9d01', '#ff8a01')

colour1 <- '#ff8a01'
colour2 <- '#1d9d01'

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('outs/', cluster_of_interest[cl], '_vs_', group2)
    dir.create(dir_path)
    
    de_genes <- FindMarkers(object = object, features = features, only.pos = FALSE, group.by = grouping, ident.1 = cluster_of_interest[cl], ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    ##prepare the plotting table (split by the log2FC and position the most negative in reverse)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    #build and save the volcano plot
    volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ]), max.overlaps = 50, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', cluster_of_interest, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_volcano_plot.pdf'), height = 8, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_de_list.csv'))
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for the positive genes
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av_wf@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.eps'), width = 5.5, height = 11)
    
    #create heatmap for the negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av_wf@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.eps'), width = 5.5, height = 11)
    
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        }
        }
    #end of the einrichr loop
}
#end of the clusters loop

## KIF2C_all vs SYTL4

In [ ]:
#create grouping 

pbmc@meta.data$h1_general_all <- 'FALSE'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general_all)

In [ ]:
unique(filter(pbmc@meta.data, h1_general_all == 'KIF2C')$s_clonotypes)
unique(filter(pbmc@meta.data, h1_general_all == 'SYTL4')$s_clonotypes)

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('SYTL4')
group2 <- 'KIF2C'
object <- pbmc
grouping <- 'h1_general_all'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]
Idents(object) <- 'h1_general'
levels(object) <- c('KIF2C', 'JU', 'SYTL4', 'FALSE')
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#for the heatmap
object_wf <- subset(object, idents = 'FALSE', invert = TRUE)
levels(object_wf) <- c('KIF2C', 'JU', 'SYTL4')
object_av_wf <- AverageExpression(object_wf, assay = "RNA", return.seurat = TRUE, verbose = FALSE)
colours_diagnosis_groups_wf <- c('#5cadfb', '#1d9d01', '#ff8a01')


colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#bebebe')
colour1 <- '#ff8a01'
colour2 <- '#12a69a'

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('outs/', cluster_of_interest[cl], '_vs_', group2, '_all')
    dir.create(dir_path)
    
    de_genes <- FindMarkers(object = object, features = features, only.pos = FALSE, group.by = grouping, ident.1 = cluster_of_interest[cl], ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    ##prepare the plotting table (split by the log2FC and position the most negative in reverse)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    #build and save the volcano plot
    volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ]), max.overlaps = 50, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', cluster_of_interest, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_volcano_plot.pdf'), height = 8, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_de_list.csv'))
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for the positive genes
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av_wf@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.eps'), width = 5.5, height = 11)
    
    #create heatmap for the negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av_wf@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.eps'), width = 5.5, height = 11)
    
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        }
        }
    #end of the einrichr loop
}
#end of the clusters loop

## KIF2C_all vs JU4

In [ ]:
#create grouping dge

pbmc@meta.data$h1_general_all <- 'FALSE'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU')] <- '4JU'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general_all)

In [ ]:
#for heatmaps
pbmc@meta.data$h1_general <- 'FALSE'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU')] <- '4JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general)

In [ ]:
unique(filter(pbmc@meta.data, h1_general_all == 'KIF2C')$s_clonotypes)
unique(filter(pbmc@meta.data, h1_general_all == '4JU')$s_clonotypes)
unique(filter(pbmc@meta.data, h1_general_all == 'SYTL4')$s_clonotypes)

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('4JU')
group2 <- 'KIF2C'
object <- pbmc
grouping <- 'h1_general_all'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]

Idents(object) <- 'h1_general'
levels(object) <- c('KIF2C', 'JU', 'SYTL4', '4JU', 'FALSE')
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#for the heatmap
object_wf <- subset(object, idents = 'FALSE', invert = TRUE)
levels(object_wf) <- c('KIF2C', 'JU', 'SYTL4', '4JU')
object_av_wf <- AverageExpression(object_wf, assay = "RNA", return.seurat = TRUE, verbose = FALSE)
colours_diagnosis_groups_wf <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff')


colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff', '#bebebe')
colour1 <- '#8800ff'
colour2 <- '#12a69a'

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('outs/', cluster_of_interest[cl], '_vs_', group2, '_all')
    dir.create(dir_path)
    
    de_genes <- FindMarkers(object = object, features = features, only.pos = FALSE, group.by = grouping, ident.1 = cluster_of_interest[cl], ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    ##prepare the plotting table (split by the log2FC and position the most negative in reverse)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    #build and save the volcano plot
    volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ]), max.overlaps = 50, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', cluster_of_interest, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_volcano_plot.pdf'), height = 8, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_de_list.csv'))
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for the positive genes
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av_wf@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.eps'), width = 5.5, height = 11)
    
    #create heatmap for the negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av_wf@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.eps'), width = 5.5, height = 11)
    
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        }
        }
    #end of the einrichr loop
}
#end of the clusters loop

## SYTL4 vs JU4

In [ ]:
#create grouping dge

pbmc@meta.data$h1_general_all <- 'FALSE'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU')] <- '4JU'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general_all)

In [ ]:
#for heatmaps
pbmc@meta.data$h1_general <- 'FALSE'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU')] <- '4JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general)

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('4JU')
group2 <- 'SYTL4'
object <- pbmc
grouping <- 'h1_general_all'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]

Idents(object) <- 'h1_general'
levels(object) <- c('KIF2C', 'JU', 'SYTL4', '4JU', 'FALSE')
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#for the heatmap
object_wf <- subset(object, idents = 'FALSE', invert = TRUE)
levels(object_wf) <- c('KIF2C', 'JU', 'SYTL4', '4JU')
object_av_wf <- AverageExpression(object_wf, assay = "RNA", return.seurat = TRUE, verbose = FALSE)
colours_diagnosis_groups_wf <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff')


colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff', '#bebebe')
colour1 <- '#8800ff'
colour2 <- '#ff8a01'

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('outs/', cluster_of_interest[cl], '_vs_', group2, '_all')
    dir.create(dir_path)
    
    de_genes <- FindMarkers(object = object, features = features, only.pos = FALSE, group.by = grouping, ident.1 = cluster_of_interest[cl], ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    ##prepare the plotting table (split by the log2FC and position the most negative in reverse)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    #build and save the volcano plot
    volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ]), max.overlaps = 50, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', cluster_of_interest, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_volcano_plot.pdf'), height = 8, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_de_list.csv'))
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for the positive genes
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av_wf@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.eps'), width = 5.5, height = 11)
    
    #create heatmap for the negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av_wf@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.eps'), width = 5.5, height = 11)
    
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        }
        }
    #end of the einrichr loop
}
#end of the clusters loop

## KIF2C_all vs JU4/JU3

In [ ]:
#create grouping dge

pbmc@meta.data$h1_general_all <- 'FALSE'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU') | startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_3JU')] <- '3or4JU'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general_all)

In [ ]:
#for heatmaps
pbmc@meta.data$h1_general <- 'FALSE'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU') | startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_3JU')] <- '3or4JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general)

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('3or4JU')
group2 <- 'KIF2C'
object <- pbmc
grouping <- 'h1_general_all'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]

Idents(object) <- 'h1_general'
levels(object) <- c('KIF2C', 'JU', 'SYTL4', '3or4JU', 'FALSE')
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#for the heatmap
object_wf <- subset(object, idents = 'FALSE', invert = TRUE)
levels(object_wf) <- c('KIF2C', 'JU', 'SYTL4', '3or4JU')
object_av_wf <- AverageExpression(object_wf, assay = "RNA", return.seurat = TRUE, verbose = FALSE)
colours_diagnosis_groups_wf <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff')


colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff', '#bebebe')
colour1 <- '#8800ff'
colour2 <- '#12a69a'

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('outs/', cluster_of_interest[cl], '_vs_', group2, '_all')
    dir.create(dir_path)
    
    de_genes <- FindMarkers(object = object, features = features, only.pos = FALSE, group.by = grouping, ident.1 = cluster_of_interest[cl], ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    ##prepare the plotting table (split by the log2FC and position the most negative in reverse)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    #build and save the volcano plot
    volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ]), max.overlaps = 50, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', cluster_of_interest, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_volcano_plot.pdf'), height = 8, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_de_list.csv'))
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for the positive genes
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av_wf@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.eps'), width = 5.5, height = 11)
    
    #create heatmap for the negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av_wf@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.eps'), width = 5.5, height = 11)
    
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        }
        }
    #end of the einrichr loop
}
#end of the clusters loop

## SYTL4 vs JU4/JU3

In [ ]:
#create grouping dge

pbmc@meta.data$h1_general_all <- 'FALSE'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'KIF2C'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU') | startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_3JU')] <- '3or4JU'
pbmc@meta.data$h1_general_all[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general_all)

In [ ]:
#for heatmaps
pbmc@meta.data$h1_general <- 'FALSE'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_KIF2C')] <- 'KIF2C'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_JU')] <- 'JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_4JU') | startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_3JU')] <- '3or4JU'
pbmc@meta.data$h1_general[startsWith(x = pbmc@meta.data$s_clonotypes, 'H1_SYTL4')] <- 'SYTL4'
unique(pbmc@meta.data$h1_general)

In [ ]:
# presort the features according to the expression within the samples of the group of interest
#define the group of analysis, cluster of interest, clustering column, patient column
cluster_of_interest <- c('3or4JU')
group2 <- 'SYTL4'
object <- pbmc
grouping <- 'h1_general_all'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]

Idents(object) <- 'h1_general'
levels(object) <- c('KIF2C', 'JU', 'SYTL4', '3or4JU', 'FALSE')
object_av <- AverageExpression(object, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

#for the heatmap
object_wf <- subset(object, idents = 'FALSE', invert = TRUE)
levels(object_wf) <- c('KIF2C', 'JU', 'SYTL4', '3or4JU')
object_av_wf <- AverageExpression(object_wf, assay = "RNA", return.seurat = TRUE, verbose = FALSE)
colours_diagnosis_groups_wf <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff')


colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#8800ff', '#bebebe')
colour1 <- '#8800ff'
colour2 <- '#ff8a01'

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    
    #create the dir with output
    dir_path <- paste0('outs/', cluster_of_interest[cl], '_vs_', group2, '_all')
    dir.create(dir_path)
    
    de_genes <- FindMarkers(object = object, features = features, only.pos = FALSE, group.by = grouping, ident.1 = cluster_of_interest[cl], ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    ##prepare the plotting table (split by the log2FC and position the most negative in reverse)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    #build and save the volcano plot
    volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:20, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:20, ]), max.overlaps = 50, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', cluster_of_interest, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_volcano_plot.pdf'), height = 8, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', cluster_of_interest[cl], '_',  '_vs_', group2, '_de_list.csv'))
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for the positive genes
    
    #create heatmap for positive genes
    if(nrow(de_positive) > 1){
    #create ordered heatmap
    ordered_genes_positive <- object_av_wf@assays$RNA@scale.data[de_positive$genes, ] 
    ordered_genes_positive <- as.data.frame(ordered_genes_positive)
    ordered_genes_positive <- ordered_genes_positive[order(-ordered_genes_positive[, cluster_of_interest[cl]]), ]
    #prepare the genes for heatmap
    ordered_genes_positive <- rownames(ordered_genes_positive)} else {ordered_genes_positive <- rownames(de_positive)} 
    
    #top 20 sorted
    if(length(ordered_genes_positive) > 30){
    ordered_genes_plot <- ordered_genes_positive[1:30]
    } else {ordered_genes_plot <- ordered_genes_positive}
    #DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7,  angle = 270, hjust = 1,
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', cluster_of_interest[cl], '_heatmap_without_false.eps'), width = 5.5, height = 11)
    
    #create heatmap for the negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 5.5, height = 11)
    
    #build vlnplot
    if(length(ordered_genes_plot) > 1){
    vlnplot <- VlnPlot(object, features = ordered_genes_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))+ 
               geom_point_rast(position = 'jitter', size = 0.01, alpha = 0.7, colour = 'grey')
    layer1 <- vlnplot$layers[[1]]
    layer2 <- vlnplot$layers[[2]]
    
    vlnplot$layers[[1]] <- layer2
    vlnplot$layers[[2]] <- layer1
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.pdf'), width = 5.5, height = 12)
    ggsave(vlnplot, file = paste0(dir_path, '/', group2, '_vlnplot.eps'), width = 5.5, height = 12)}
    
    #create heatmap for negative genes
    
    if(nrow(de_negative) > 1){
    #create ordered heatmap
    ordered_genes_negative <- object_av_wf@assays$RNA@scale.data[de_negative$genes, ] 
    ordered_genes_negative <- as.data.frame(ordered_genes_negative)
    ordered_genes_negative <- ordered_genes_negative[order(-ordered_genes_negative[, group2]), ]
    
    #prepare the genes for heatmap
    ordered_genes_negative <- rownames(ordered_genes_negative)} else {ordered_genes_negative <- rownames(de_negative)}
    
    #top 30 sorted
    if(length(ordered_genes_negative) > 30){
    ordered_genes_plot <- ordered_genes_negative[1:30]
    } else {ordered_genes_plot <- ordered_genes_negative}
    heatmap <- DoHeatmap(object_av_wf, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups_wf) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.pdf'), width = 5.5, height = 11)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_without_FALSE.eps'), width = 5.5, height = 11)
    
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(cluster_of_interest[cl], databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', cluster_of_interest[cl], '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 5, height = 11)
        }
        }
    #end of the einrichr loop
}
#end of the clusters loop

# Save h5ad file for the diffusion maps analysis

In [ ]:
levels(pbmc)
object <- subset(pbmc, idents = c('MAIT_1', 'MAIT_2'), invert = TRUE)
DefaultAssay(object) <- 'integrated'

In [ ]:
levels(object)

In [ ]:
library('SeuratData')
library('SeuratDisk')
SaveH5Seurat(object, filename = "pbmc_subset_fordiffusion.h5Seurat")
Convert("pbmc_subset_fordiffusion.h5Seurat", dest = "h5ad")

# Diffusion map: in Python

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import scipy as sp
import random

In [4]:
import numpy as np
import umap

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150, frameon=False, figsize=(7, 7), facecolor='white')

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
import matplotlib

## Generate diffusion maps

In [ ]:
#load the object
random.seed(0)
adata = sc.read_h5ad("./pbmc_subset_fordiffusion.h5ad")

In [ ]:
adata

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes = 1000, flavor="seurat")
sc.tl.pca(adata)
#adata.obs["cd8names"] = adata.obs["cd8names"].apply(str)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=18)

In [ ]:
sc.tl.diffmap(adata)

In [ ]:
palette = ["#a6cee3", "#1e78b4", "#b2df8a", "#34a12d", "#fa9a99", "#e31a1c", "#fcbf6f", "#ff7f01", "#cab2d6"]

In [ ]:
%matplotlib inline
sc.set_figure_params(dpi=300, frameon=False, figsize=(10, 3), facecolor='white')
sc.pl.diffmap(adata,components=[1,2],color=["cd8names"], frameon=True, projection = "2d",
              legend_loc="right margin", palette = palette, legend_fontsize = 12, legend_fontweight = 'normal', size = 20,
             save = '_withlegend.pdf')

sc.set_figure_params(dpi=300, frameon=False, figsize=(9, 3), facecolor='white')
sc.pl.diffmap(adata,components=[1,2],color=["cd8names"], frameon=True, projection = "2d",
              legend_loc="on data", palette = palette, legend_fontsize = 7, legend_fontweight = 'normal', size = 20,
             save = '.pdf')

## Add the pseudotime score

In [ ]:
#annotate the root and compute pseudotime
adata.uns['iroot'] = np.flatnonzero(adata.obs['cd8names'] == '1_CCR7')[0]
sc.tl.dpt(adata)

In [ ]:
#visualize the new pseudotime
sc.set_figure_params(dpi=300, frameon=False, figsize=(10, 3), facecolor='white', vector_friendly = True)
sc.pl.diffmap(adata,components=[1,2],color=["dpt_pseudotime"], frameon=True, projection = "2d",
              legend_loc="on data", color_map = 'binary', legend_fontsize = 12, legend_fontweight = 'normal', size = 20, sort_order = True, save = '_withdm_pseudotime_bw.pdf')